In [ ]:
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
from nidaqmx.stream_writers import AnalogMultiChannelWriter
import numpy as np

import tifffile as tiff
from pathlib import Path
from scipy import fft

import time

import matplotlib.pyplot as plt

### Calibration

In [ ]:
cal_task = Task()
cal_task.ao_channels.add_ao_voltage_chan("Dev2/ao0:3", min_val=-10, max_val=10)
# cal_task.timing.cfg_samp_clk_timing(
#     rate=40000,
#     source="OnboardClock",
#     active_edge=Edge.RISING,
#     sample_mode=AcquisitionType.CONTINUOUS,
#     #sample_mode=AcquisitionType.FINITE,
#     samps_per_chan=1,
# )

cal_task_xy = Task()
cal_task_xy.ao_channels.add_ao_voltage_chan("Dev5/ao2", min_val=-10, max_val=10)
cal_task_xy.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    #sample_mode=AcquisitionType.FINITE,
    samps_per_chan=10000,
)

In [ ]:
xy_val = 2
xy = np.linspace(-xy_val,xy_val,200)
for i in range(50):
    xy = np.append(xy, np.linspace(xy_val,-xy_val,200))
    xy = np.append(xy, np.linspace(-xy_val,xy_val,200))

array = xy
cal_task_xy.write(array, auto_start=True)

In [ ]:
piezo_val = np.linspace(1, 4, 5)
possible_z_val = np.linspace(-1, 1,30)#30)

xy_val = 3
xy = np.linspace(-xy_val,xy_val,100)
xy = np.append(xy, np.linspace(xy_val,-xy_val,100))
for i in range(39):
    xy = np.append(xy, np.linspace(-xy_val,xy_val,100))
    xy = np.append(xy, np.linspace(xy_val,-xy_val,100))
    
cam = np.zeros(np.shape(xy))
cam[10] = 5

# counter = 0                                         
start_time = time.time_ns()
for piezopos in piezo_val:
    # set piezopos
    # p = np.ones(np.shape(xy)) * piezopos
    for zgalvopos in possible_z_val:
        
        # print(counter)
        # counter +=1
        
        cam_pulse = 5
  
        # set z galvo
        # z = np.ones(np.shape(xy)) * zgalvopos

        # run and get camera image
        cal_array = np.stack([0, zgalvopos, piezopos, cam_pulse])
        cal_task.write(cal_array, auto_start=True)
        
        time.sleep(0.1)
        cam_pulse = 0
        cal_array = np.stack([0, zgalvopos, piezopos, cam_pulse])
        cal_task.write(cal_array, auto_start=True)
        
        # cal_task_xy.write([xy], auto_start=True)


print((time.time_ns() - start_time)*1e-9)

In [ ]:
print(np.shape(xy))
cal_task_xy.stop()

In [ ]:
# z_writer = AnalogMultiChannelWriter(cal_task.out_stream)

In [ ]:

cam = np.zeros(np.shape(xy))
cam[10] = 5
            
print(np.count_nonzero(cam == 5))
print(np.shape(xy))


# for i in range(3):
counter = 5
cam[10] = 5
while counter > 0:
    print(counter)
    counter -= 1
    cal_array = np.stack([xy, z, p, cam])
    # print(np.shape(cal_array))
    cal_task.write(cal_array,auto_start=True)
    cam[10]= 0
    # cam[10] = 5 if counter == 2 else 0

# time.sleep(1)
    
cal_task.stop()

In [ ]:
master = Path(r"C:\Users\Admin\Documents")
path = master / "fish_neurod_tdtomato_gfpneuromast_red_greenseparate(4).tif"
stack = tiff.imread(path)
print(np.shape(stack))
nx, ny = np.shape(stack)[1:]

In [ ]:
z_val = np.zeros((np.shape(piezo_val)[0], np.shape(possible_z_val)[0]))
optimal_z = np.zeros(np.shape(piezo_val))
print(np.shape(z_val))
count = 0

for piezopos in range(np.shape(piezo_val)[0]):
    for zgalvopos in range(np.shape(possible_z_val)[0]):
        # load tiffs and get fft 
        img = stack[count]
        img_fft = abs(fft.fftshift(fft.fft2(img)))
        
        # choose the best fft
        z_val[piezopos, zgalvopos]  = np.mean(img_fft[0:200] + np.mean(img_fft[:, 0:200]))
        count += 1
    optimal_z[piezopos] = np.argmax(z_val[piezopos])

In [ ]:
optimal_z

In [ ]:
path = Path(r"C:\Users\Admin\Documents\mpImage(28).tif")
stack = tiff.imread(path)

In [ ]:
img = np.mean(stack, 0)
img_fft = fft.fftshift(fft.fft2(img))

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img, vmin=0, vmax=5000)
ax[1].imshow(np.abs(img_fft), vmin=0, vmax=50000)

In [ ]:
task.write(array, auto_start=True)

### Acquisition

In [ ]:
task = Task()
task.ao_channels.add_ao_voltage_chan("Dev2/ao0:3", min_val=-10, max_val=10)
task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)

In [ ]:
xy_val = 3
xy = np.linspace(-xy_val,xy_val,200)
t =  np.zeros(200)
t[0] = 5
t0 = t.copy()
for i in range(50):
    xy = np.append(xy, np.linspace(xy_val,-xy_val,200))
    xy = np.append(xy, np.linspace(-xy_val,xy_val,200))
    t = np.append(t, t0)
    t = np.append(t, t0)
    
#xy = np.concatenate([np.linspace(-1,1,200), np.linspace(1,-1,200)])
z = np.linspace(-1, 1, 200*101)
p = np.linspace(5, 0, 200*100 - 800)
p = np.append(p, np.linspace(0, 5, 1000))
              
print(np.shape(z))
print(np.shape(p))
#p = np.linspace(5, 0, 200*101)
array = np.stack([xy, z, p, t])

#plt.figure()
#plt.plot(array.T)
#plt.xlim(0,500)

In [ ]:
task.write(array, auto_start=True)

In [ ]:
task.stop()

In [ ]:
path = Path(r"C:\Users\Admin\Documents\mpImage(61).tif")
stack = tiff.imread(path)

In [ ]:
np.shape(stack)

In [ ]:
piezo_calibration = np.asarray([piezo1, piezo2, piezo3])
z_calibration = np.asarray([z_galvo1, z_galvo2, z_galvo3])

In [ ]:
calibration_data = np.zeros((3,3))
calibration_data[:, 0] = piezo_calibration.T
calibration_data[:, 1] = z_calibration.T
piezo_val = np.pad(
    calibration_data[:, 0:1],
    ((0, 0), (1, 0)),
    constant_values=1.0,
    mode="constant",
)
galvo = calibration_data[:, 1]

In [ ]:
# solve least squares according to standard formula b = (XtX)^-1 * Xt * y
piezo_cor = np.linalg.pinv(piezo_val.T @ piezo_val)
calibration = piezo_cor @ piezo_val.T @ galvo

In [ ]:
calibration